#### Directory Path

In [1]:
import os

In [2]:
# Current folder path
%pwd

'/Volumes/Macintosh/mlflow-dvc-food-spoilage-detector/notebooks'

In [3]:
# Go to the roor directory and give the path
os.chdir("../")
%pwd

'/Volumes/Macintosh/mlflow-dvc-food-spoilage-detector'

### Initial Model

In [4]:
from dataclasses import dataclass
from pathlib import Path
from constants import *
from utils.base_utils import read_yaml, create_directories

In [5]:
# == Entity ==
@dataclass(frozen=True)
class PrepareBaseModelConfig:
   root_dir: Path
   base_model_path: Path
   updated_base_model_path: Path
   params_image_size: list
   params_learning_rate: float
   params_include_top: bool
   params_weights: str
   param_classes: int

In [6]:
# Configuration Manager ==
class ConfigurationManager:

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([self.config.artifacts_root])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path= Path(config.updated_base_model_path),
            params_image_size= self.params.IMAGE_SIZE,
            params_learning_rate= self.params.LEARNING_RATE,
            params_include_top= self.params.INCLUDE_TOP,
            params_weights= self.params.WEIGHTS,
            param_classes= self.params.CLASSES

        )
        return prepare_base_model_config

In [7]:
# == Componets ==

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [8]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    def update_base_model(self):
        self.full_model = self._prepare_full_modle(
            model=self.model,
            classes=self.config.param_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
            )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    
    
    @staticmethod
    def _prepare_full_modle(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif(freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
               layer.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs= model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=[tf.keras.metrics.Accuracy()]
        )

        full_model.summary()
        return full_model
    
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

2025-08-07 20:49:06,922 | fsd_logger | INFO | base_utils.py:39 | YAML file config/config.yaml loaded successfully
2025-08-07 20:49:06,924 | fsd_logger | INFO | base_utils.py:39 | YAML file params.yaml loaded successfully
2025-08-07 20:49:06,926 | fsd_logger | INFO | base_utils.py:63 | Directory created at artifacts
2025-08-07 20:49:06,926 | fsd_logger | INFO | base_utils.py:63 | Directory created at artifacts
2025-08-07 20:49:07,189 | tensorflow | WARNING | saving_utils.py:359 | Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 244, 64)      1792      
                    

/opt/anaconda3/envs/fsd-env/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
